### Imports

In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore') # hopefully nothing explodes

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
print(tf.version)
print(tf.config.list_physical_devices('GPU'))

2023-12-13 01:50:10.020950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 01:50:10.021016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 01:50:10.023198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 01:50:10.040315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 01:50:10.921772: W tensorflow/compiler/tf2

<module 'tensorflow._api.v2.version' from '/home/zyzz/anaconda3/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-12-13 01:50:12.667107: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 01:50:12.779553: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 01:50:12.779605: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
# For reproducible results
os.environ['PYTHONHASHSEED']=str(42)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)  

## Data

In [3]:
# Constants
val_size = 0.2
data_path = 'training_dataset'
seq_length = 26      # predictions based on previous seq_length data entries
forecast_length = 9  # predicting forecast_length time steps into the future
sample_length = seq_length + forecast_length

In [4]:
# Read data (ignore categories)
training_data = np.load(os.path.join(data_path, 'training_data.npy'))
valid_periods = np.load(os.path.join(data_path, 'valid_periods.npy'))

# Filter out unvalid data
data = []
for i, row in enumerate(training_data):
    data.append(row[valid_periods[i][0]:valid_periods[i][1]])

print(f"({len(data)}, -)", valid_periods.shape)

(48000, -) (48000, 2)


In [5]:
# Convert time series to {x: sequences of length seq_length, y: values to be predicted from previous sequence}
def to_sequences(time_series):
    
    x = []
    y = []
    
    for i in range(time_series.shape[0]-seq_length-forecast_length+1):
        x.append(time_series[i:i+seq_length])
        y.append(time_series[i+seq_length:i+seq_length+forecast_length])  
    
    x = np.array(x)
    y = np.array(y)
    
    return {'x': x.reshape((x.shape[0], x.shape[1], -1)), 'y': y.reshape((y.shape[0], y.shape[1], -1))}

# Shuffle data
np.random.shuffle(data)
 
# Build sequences from the non-correlated time series, and append them to corresponding data set
# Note: there is no overlap between train and validation; each processed time series is used in train xor val
X_train, X_val = [], []
y_train, y_val = [], []
split_index = int((1-val_size)*len(data))
for i, time_series in enumerate(data): 
    if (len(time_series) >= sample_length): # assert we can draw at least one sample from the time_series
        sequences = to_sequences(time_series)
        if(i < split_index):
            X_train.append(sequences['x']) 
            y_train.append(sequences['y'])   
        else:
            X_val.append(sequences['x']) 
            y_val.append(sequences['y'])  

# Convert lists to nparrays 
X_train = np.concatenate(X_train, axis=0)
X_val = np.concatenate(X_val, axis=0)
y_train = np.concatenate(y_train, axis=0)
y_val = np.concatenate(y_val, axis=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(6307286, 26, 1) (1579211, 26, 1) (6307286, 9, 1) (1579211, 9, 1)


## ML

In [6]:
input_size = (X_train.shape[1], X_train.shape[2])
batch_size = 128
epochs = 1000
dout = 0.15

model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(96, input_shape=input_size),
    tf.keras.layers.Dropout(rate=dout),      # prevent overfitting
    tf.keras.layers.Dense(forecast_length)   # output is next forecast_length values
])

# Evaluate using MSE as loss function and the adam optimizer
model.compile(optimizer='adam', loss='mse')

# Stop training when validation loss stops improving, maintain best weights
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,         # how many epochs to check for improvement before stopping
    restore_best_weights=True,
)

history = model.fit(X_train,
                    y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(X_val, y_val),
                    callbacks=callback,
                    verbose=1)

# Evaluate on validation data
val_result = model.evaluate(X_val, y_val, verbose=0)
print(f"Val loss (MSE): {val_result}")

2023-12-13 01:50:25.259386: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 01:50:25.259509: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 01:50:25.259543: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 01:50:25.449849: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 01:50:25.449987: I external/local_xla/xla/stream_executor

Epoch 1/1000


2023-12-13 01:50:29.911553: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-13 01:50:30.862084: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f40750d2090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-13 01:50:30.862122: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2023-12-13 01:50:30.868063: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702428630.944052   67791 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49276/49276 [==============================] - 291s 6ms/step - loss: 0.0091 - val_loss: 0.0080
Epoch 2/1000
49276/49276 [==============================] - 290s 6ms/step - loss: 0.0083 - val_loss: 0.0078
Epoch 3/1000
49276/49276 [==============================] - 285s 6ms/step - loss: 0.0081 - val_loss: 0.0077
Epoch 4/1000
49276/49276 [==============================] - 288s 6ms/step - loss: 0.0080 - val_loss: 0.0076
Epoch 5/1000
49276/49276 [==============================] - 282s 6ms/step - loss: 0.0080 - val_loss: 0.0076
Epoch 6/1000
49276/49276 [==============================] - 277s 6ms/step - loss: 0.0079 - val_loss: 0.0076
Epoch 7/1000
49276/49276 [==============================] - 276s 6ms/step - loss: 0.0079 - val_loss: 0.0075
Epoch 8/1000
49276/49276 [==============================] - 277s 6ms/step - loss: 0.0079 - val_loss: 0.0075
Epoch 9/1000
49276/49276 [==============================] - 275s 6ms/step - loss: 0.0078 - val_loss: 0.0075
Epoch 10/1000
49276/49276 [==============

In [7]:
# Val loss for each prediction step
y_pred = model.predict(X_val)
for t in range(forecast_length):
    m = tf.keras.metrics.MeanSquaredError()
    m.update_state(y_val[:, t], y_pred[:, t])
    print(f'Val loss (MSE) {t+1}h forward: {m.result().numpy()}')

49351/49351 [==============================] - 83s 2ms/step
Val loss (MSE) 1h forward: 0.0035941225942224264
Val loss (MSE) 2h forward: 0.004795367829501629
Val loss (MSE) 3h forward: 0.005801919382065535
Val loss (MSE) 4h forward: 0.00680383900180459
Val loss (MSE) 5h forward: 0.007624762132763863
Val loss (MSE) 6h forward: 0.008407504297792912
Val loss (MSE) 7h forward: 0.00920922216027975
Val loss (MSE) 8h forward: 0.009970426559448242
Val loss (MSE) 9h forward: 0.010697007179260254


In [8]:
# save model
model.save('GRU_v3')

INFO:tensorflow:Assets written to: GRU_v3/assets


INFO:tensorflow:Assets written to: GRU_v3/assets
